# Data prep for Pump it Up project

Back in the pcda class we did quite a bit of data prep work using R. Rarely is data ready for statistical/ML predictive modeling in its raw state. In this notebook we will do a few common data prep tasks in Python. This is not meant to be totally comprehensive and I make a bunch of big assumptions and take some shortcuts so that we can move on to modeling. A few of things we'll do, include:

* reading in the raw data from csv files - pandas
* automated EDA - pandas profiling and sweetviz
* manual EDA - pandas and matplotlib/Seaborn
* data type conversions - pandas
* variable dropping - pandas
* factor lumping - a port of R forcats package

## Preliminaries

In [ ]:
# To auto-reload modules in jupyter notebook (so that changes in files *.py doesn't require manual reloading):
# https://stackoverflow.com/questions/5364050/reloading-submodules-in-ipython
%load_ext autoreload
%autoreload 2

Import commonly used libraries and magic command for inline plotting

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
%matplotlib inline

## Read in raw data

Let's do the following:

* read in train_x and train_y to pandas dataframes,
* merge the data frames into a single training dataframe.

In [ ]:
train_x = pd.read_csv("./data/raw/train_x.csv", parse_dates=['date_recorded'])
train_y = pd.read_csv("./data/raw/train_y.csv")

We can join the two dataframes on the `id` column.

In [ ]:
train_df = pd.merge(train_x, train_y, on='id')

## Initial EDA

As always, let's check out the structure of the dataframes and scan the values a bit.

In [ ]:
train_x.info()

In [ ]:
train_y.info()

In [ ]:
train_df.info()

In [ ]:
train_df.iloc[:5, :15]

In [ ]:
train_df.iloc[:5, 15 :30]

In [ ]:
train_df.iloc[:5, 30:]

Since we will likely be dropping one or more columns before trying to fit models, let's create a list of all the column names and then we can remove some as we go. We can use `train_df.columns` to get an `Index` object whose values are the column names. To get a list version, use the `tolist` method.

In [ ]:
cols_to_keep = train_df.columns.tolist()
cols_to_keep

Let's drop `date_recorded` and `recorded_by` as they seem irrelevant for a predictive model. What are some different ways to remove an item from a list in Python?

In [ ]:
# To remove a single element
# cols_to_keep.remove('date_recorded')

# To remove multiple elements, create a list of cols to drop and then use
# a list comprehension to get a list of the ones to keep
to_drop = ['date_recorded', 'recorded_by']

cols_to_keep = [c for c in cols_to_keep if c not in to_drop]
cols_to_keep

## Automated EDA

In R, we used a package called skimr to do quick, automated, basic EDA. Surely similar tools must exist for Python? A [little digging turned up several tools](https://github.com/mstaniak/autoEDA-resources). We'll look at two here - pandas-profiling and sweetviz.

### ydata-profiling (formerly pandas-profiling)

This has been around for a while and quickly gives you a much deeper look at pandas dataframes than you'd get from the the `info` and `describe` methods. The main site is at https://github.com/ydataai/ydata-profiling. Earlier this year, the very popular **pandas-profiling** package became **ydata-profiling** and has a much broader goal than the original.

If you want to install it and try it out, I'd suggest using conda to install it via:

    conda install -c conda-forge ydata-profiling
    
I haven't had time to test out this new version. Here's a screenshot from the old version and I've left the old report in the `output/` folder.


![pandas profiling output](images/pandas_profiling_output.PNG)

### Sweetviz

Another impressive automated EDA packack is [Sweetviz](https://github.com/fbdesignpro/sweetviz. Like pandas-profiling, it creates an interactive HTML based report. It includes some features aimed at predictive modeling such as explicit analysis of the target variable and advanced correlation analysis. Some relevant references on Python tools for correlation analysis include:

* https://towardsdatascience.com/the-search-for-categorical-correlation-a1cf7f1888c9
* https://towardsdatascience.com/better-heatmaps-and-correlation-matrix-plots-in-python-41445d0f2bec

To install sweetviz:

    pip install sweetviz
    
I've decided not to install it this year as development on the package seems to have slowed or stopped. Again, I've kept the old report around so you can see it. It's in the `output/` folder.



In [ ]:
#import sweetviz

In [ ]:
#report = sweetviz.analyze(train_df)

In [ ]:
#report.show_html("output/sweetviz_report.html")

From the data dictionary and the dataframe snooping above, a few observations:

* the target variable, `status_group`, has three levels
* many fields are non-numeric
* several fields have a large proportion of zero values - what is the meaning (missing or zero)?
* some of the numeric fields are actually categorical data (`region_code` and `district_code`)
* some of the fields seem to be related in a hierarchical fashion (`extraction_type`, `extraction_type_group`, `extraction_type_class`)
* Seems like we could eliminate a number of variables before model building.

### Target variable - status_group

While this information is available in the automated EDA reports, just demonstrating some basic pandas techniques.

In [ ]:
train_df.groupby(['status_group']).size()

In [ ]:
train_df['status_group'].value_counts(normalize=True)

### Exploring the numeric variables
Sometimes it's useful to be able to select columns by data type. For example to get a list of numeric columns:

In [ ]:
train_df.select_dtypes(include=np.number).columns.tolist()

The `id` field is just an identifier and won't be used in any models but we'll leave it in the dataframe for now.

The `amount_tsh` field has about 70% zeroes and not exactly sure what this means. 

The `gps_height` field has about 30% zeroes. Is this really sea level or missing data?

The `latitude` and `longitude` fields are fully populated. Each degree is about 70 miles.

The `num_private` has no definition posted and is 99% zeroes. For the non-zeroes, let's see how the target variable is distributed. Note the use of the `value_counts` function to avoid having to do a group by and using `normalize=True` to get percentages instead of counts. See https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.value_counts.html.

In [ ]:
train_df[train_df.num_private > 0]['status_group'].value_counts(normalize=True)

Higher percentage of functional wells in the subset of data having `num_private > 0`. Let's keep it around.

Both `region_code` and `district_code` are categorical. Let's convert their type.

In [ ]:
train_df["region_code"] = train_df["region_code"].astype("category")
train_df["district_code"] = train_df["district_code"].astype("category")

The `population` field has 36% zeroes. This is defined as the population around the well. Not sure exactly what the 0's mean.

In [ ]:
train_df.groupby('construction_year')['status_group'].value_counts(normalize=True)

In [ ]:
status_by_year = pd.crosstab(train_df['construction_year'], train_df['status_group'], normalize='index')
status_by_year

Not surprisingly, looks like newer wells are more likely to be functioning.

Can use pandas `plot` function to quickly generate plot for this dataframe. It's just using matplotlib under the hood.

In [ ]:
status_by_year[status_by_year.index > 0].plot(kind='line')

### Exploring the categorical variables
Categorical variables provide challenges in building predictive models in sklearn. For example, when we built regression models in R, we simply included categorical variables in our regression formulas and R automatically created dummy (binary indicator) variables. In sklearn, we have to do this work ourself. Similarly, the implementation of random forests in the R randomForest package can handle categorical variables with no preprocessing necessary.
Thankfully, sklearn has a number of tools to help you properly encode such variables for different models and algorithms. We will explore this in more detail in the next notebook when we build models. For now, check out the following blog post that address this topic.

* https://pbpython.com/categorical-encoding.html from Practical Business Python




Review the EDA reports to look for:

* obvious variables to drop due to things like little to no variance in the variable values.
* variables that might be candidates for lumping due to huge numbers of unique values
* categorical variables that are related to each other - e.g. hierarchical relationships such as `extraction_type`, `extraction_type_group`, `extraction_type_class`

<div class="alert alert-block alert-info">
<b>SIDEBAR:</b> How does one do dplyr type chaining with pandas?
</div>

When exploring categorical data with pandas, sometimes I find myself longing for the R package, dplyr. For example how to do this dplyr type analysis in pandas?

    train %>% 
      group_by(payment) %>% 
      summarize(
        num_installs = n(),
        pct_functional = sum(status_group == 'functional') / num_installs
      ) %>% 
      filter(num_installs > 100) %>%
      arrange(desc(pct_functional))

To get:

    payment  num_installs   pct_functional
    pay annually	3642	0.7523339	
    pay per bucket	8985	0.6777963	
    ...
    
More generally, how to visualize the breakdown of the categorical target variable within levels of some categorical predictor? Is it just a heat map? Or entropy or gini by level in comparison (perhaps weighted by number of observations in the level) to entropy/gini of target?

Down the rabbit hole with the following reddit post. Good stuff found in there and a generally useful discussion. More work going on in adding tidyverse approaches to pandas Python.

* https://www.reddit.com/r/datascience/comments/ltkt9s/r_is_far_superior_to_python_for_data_manipulation/
* https://tomaugspurger.github.io/modern-1-intro
* https://tomaugspurger.github.io/method-chaining - this is really good
* https://stmorse.github.io/journal/tidyverse-style-pandas.html - and so is this
* [R to Python data wrangling snippets](https://gist.github.com/conormm/fd8b1980c28dd21cfaf6975c86c74d07)
* https://www.tidymodels.org/
* https://pyjanitor.readthedocs.io/
* https://towardsdatascience.com/the-unreasonable-effectiveness-of-method-chaining-in-pandas-15c2109e3c69
* https://github.com/machow/siuba

<div class="alert alert-block alert-info">
<b>END OF SIDEBAR</b> Be careful before checking out the reddit discussion above as it can become a time sink (though an interesting and useful one).
</div>

### Geographic categorical variables

There are several categorical variables that are geographic in nature.

* `wpt_name` - waterpoint name (if exists)
* `basin` - Geographic water basin
* `subvillage` - Geographic location
* `region` - Geographic location
* `region_code` - Geographic location (coded)
* `district_code` - Geographic location (coded)
* `lga` - Geographic location
* `ward` - Geographic location

In [ ]:
geo_cols = ['wpt_name', 'basin', 'subvillage', 'region', 'region_code', 'district_code', 'lga', 'ward']

train_df.loc[:, geo_cols].describe()

Let's drop `region_code` and just use `region`. Let's also remove `wpt_name`, `subvillage` and `ward` as too detailed.

In [ ]:
cols_to_keep.remove('region_code')
cols_to_keep.remove('wpt_name')
cols_to_keep.remove('subvillage')
cols_to_keep.remove('ward')

In [ ]:
train_df = train_df.loc[:, cols_to_keep]
train_df.info()

### Hierarchical variables

* `scheme_name`
  `scheme_management`
  
* `extraction_type`
  `extraction_type_group`
  `extraction_type_class`

* `management`
  `management_group`
 
* `payment`
  `payment_type`
 
* `water_quality`
  `quality_group`
 
* `quantity`
  `quantity_group`
 
* `source`
  `source_type`
  `source_class`
 
* `waterpoint_type`
  `waterpoint_type_group`
 
We should explore these more deeply, but for now, let's just include the most aggregate version of each variable.

In [ ]:
aggs_to_drop = ['extraction_type', 'extraction_type_group', 'management', 'payment', 'water_quality',
               'quantity', 'source', 'source_type', 'waterpoint_type', 'scheme_name']

cols_to_keep = [c for c in cols_to_keep if c not in aggs_to_drop]
cols_to_keep

train_df = train_df.loc[:, cols_to_keep]
train_df.info()

### Missing data
Will just fill missing data in the categorical fields with "missing".

In [ ]:
train_df['funder'] = train_df['funder'].fillna("missing")
train_df['installer'] = train_df['installer'].fillna("missing")
train_df['public_meeting'] = train_df['public_meeting'].fillna("missing")
train_df['scheme_management'] = train_df['scheme_management'].fillna("missing")
train_df['permit'] = train_df['permit'].fillna("missing")

train_df.info()

### More on categorical data

In [ ]:
categorical_cols = train_df.select_dtypes(include=['object', 'category']).columns.tolist()
categorical_cols

In [ ]:
train_df.loc[:, categorical_cols].describe()

What options exist for category lumping in Python? I'm thinking of the forcats package in R. Can we do useful lumping with `funder` and `installer`? These feel like they might have decent predictive value.

The [siuba](https://github.com/machow/siuba) package is a Python port of dplyr. The author implemented forcats as a module in that package and is now moving it to its own package. See https://pypi.org/project/forcats-py/#description. I pip installed siuba to check it out. There's **NO** need for you to do so unless you want to try it out for yourself. I'll be providing the pre-processed dataset based on this data prep notebook.

In [ ]:
#from siuba.dply.forcats import fct_lump

In [ ]:
#fct_lump(train_df['funder'], n=10).value_counts()

In [ ]:
#fct_lump(train_df['installer'], n=10).value_counts()

Let's do this for now.

In [ ]:
#train_df['funder'] = fct_lump(train_df['funder'], n=10)
#train_df['installer'] = fct_lump(train_df['installer'], n=10)
#train_df['scheme_management'] = fct_lump(train_df['scheme_management'], n=10)

#train_df.loc[:, categorical_cols].describe()

In [ ]:
#train_df.info()

## Before moving on to model building

The work we did above is pretty typical data prep work to get ready to build predictive models. However, Jupyter notebooks such as this one, while great for documenting our thinking and exploration, is not that great for automated workflows. So, I'll typically extract the relevant code and create a data prep script that I can rerun as needed to generate preprocessed data files for the next stage in our analysis. See `data_prep.py` for the code I ended up with for this dataset. You'll see that it ends with exporting to both csv and json files since I wanted to experiment with both formats.

Again, you do **NOT** need to run it. I've created the files already.

In [ ]:
# %load data_prep.py
import pandas as pd
from siuba.dply.forcats import fct_lump


train_x = pd.read_csv("./data/raw/train_x.csv", parse_dates=['date_recorded'])
train_y = pd.read_csv("./data/raw/train_y.csv")
test_x = pd.read_csv("./data/raw/test_x.csv", parse_dates=['date_recorded'])

dict_df = {"train_x": train_x,
           "test_x": test_x}
           
for key in dict_df:

    df = dict_df[key]

    # Initialize list of columns to keep
    cols_to_keep = df.columns.tolist()

    to_drop = ['date_recorded', 'recorded_by']
    cols_to_keep = [c for c in cols_to_keep if c not in to_drop]

    # Geo cols to remove
    cols_to_keep.remove('region_code')
    cols_to_keep.remove('district_code')
    cols_to_keep.remove('wpt_name')
    cols_to_keep.remove('subvillage')
    cols_to_keep.remove('ward')

    # Aggregated cols to remove
    aggs_to_drop = ['extraction_type', 'extraction_type_group', 'management', 'payment', 'water_quality',
                   'quantity', 'source', 'source_type', 'waterpoint_type', 'scheme_name']

    cols_to_keep = [c for c in cols_to_keep if c not in aggs_to_drop]

    df = df.loc[:, cols_to_keep]


    # Change booleans to strings
    df["permit"] = df["permit"].astype("string")
    df["public_meeting"] = df["public_meeting"].astype("string")

    # Missing data - code as "missing" for categorical data
    df['funder'] = df['funder'].fillna("missing")
    df['installer'] = df['installer'].fillna("missing")
    df['public_meeting'] = df['public_meeting'].fillna("missing")
    df['scheme_management'] = df['scheme_management'].fillna("missing")
    df['permit'] = df['permit'].fillna("missing")


    # Factor level lumping

    df['funder'] = fct_lump(df['funder'], n=10)
    df['installer'] = fct_lump(df['installer'], n=10)
    df['scheme_management'] = fct_lump(df['scheme_management'], n=10)
    
    df.to_json(f"data/{key}.json", orient='records')
    df.to_csv(f"data/{key}.csv", index=False)


## Further exploration
In the process of putting this notebook together, I ran across numerous interesting and relevant Python packages. I'll mention two of them and you can explore them if you are interested.


### HoloViews

This is another data visualization package. It has a different philosophy than packages like matplotlib and Seaborn (at least that's what they say):

> HoloViews helps you understand your data better, by letting you work seamlessly with both the data and its graphical representation.

* https://holoviews.org/index.html
* https://holoviews.org/getting_started/Introduction.html

### Pyvtreat

In the pcda class, one of the textbooks we use is *Practical Data Science with R* by Mount and Zumel. They created an R package called vtreat for preparing data for predictive modeling. Now it appears they've ported to Python.

* https://github.com/WinVector/pyvtreat